## 4.1 模型构造

### 4.1.1 继承Module类来构造模型

Module 类是 nn 模块里提供的一个模型构造类，是所有神经网络模块的基类，可以继承它来定义想要的模型

重载了Module类的 `__init__` 函数和 `forward` 函数

- `__init__()` 创建模型参数
- `forward()` 定义前向计算即正向传播
- 无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的backward函数

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用 MLP 父类 Module 的构造函数来进行必要的初始化。这样在构造 b 实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数 params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层


    # 定义模型的前向计算，即如何根据输入 x 计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

以上的 ML P类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的 **backward** 函数

In [18]:
X = torch.rand(2, 784)
net = MLP()
print(net)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 4.1.2 Module的子类

#### 4.1.2.1 Sequential类

当模型的前向计算为简单串联各个层的计算时，Sequential 类可以通过更加简单的方式定义模型。它可以接收一个子模块的有序字典（OrderedDict）或者一系列子模块作为参数来逐一添加 Module 的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算

In [2]:
net = nn.Sequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


#### 4.1.2.2 ModuleList类

ModuleList 接收一个子模块的列表作为输入，然后也可以类似 List 那样进行 append 和 extend 操作

In [3]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) # # 类似List的append操作
print(net[-1])  # 类似List的索引访问
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


Sequential 和 ModuleList 二者区别

- ModuleList 仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序（所以不用保证相邻层的输入输出维度匹配）
- ModuleList 没有实现 forward 功能, 执行 net(torch.zeros(1, 784)) 会报 NotImplementedError
- Sequential 内的模块需要按照顺序排列，要保证相邻层的输入输出大小相匹配, 内部 forward 功能已经实现

#### 4.1.2.3 ModuleDict类

ModuleDict接收一个子模块的字典作为输入, 然后也可以类似字典那样进行添加访问操作

In [4]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


和ModuleList一样，ModuleDict实例仅仅是存放了一些模块的字典，并没有定义forward函数需要自己定义

### 4.1.3 构造复杂的模型

In [5]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

### 小结

- 可以通过继承Module类来构造模型。
- Sequential、ModuleList、ModuleDict类都继承自Module类。
- 与Sequential不同，ModuleList和ModuleDict并没有定义一个完整的网络，它们只是将不同的模块存放在一起，需要自己定义forward函数。
- 虽然Sequential等类可以使模型构造更加简单，但直接继承Module类可以极大地拓展模型构造的灵活性。

## 4.2 模型参数的访问、初始化和共享

In [6]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


### 4.2.1 访问模型参数

通过 Module 类的 parameters() 或者 named_parameters 方法来访问所有参数（以迭代器的形式返回）, named_parameters 除了返回参数 Tensor 外还会返回其名字

In [8]:
print(net)
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


通过方括号 $[]$ 来访问网络的任一层。索引 0 表示隐藏层为 Sequential 实例最先添加的层。

In [9]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


Parameter 会自动被添加到模型的参数列表里

In [10]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


可以根据data来访问参数数值，用grad来访问参数梯度

In [11]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

tensor([[ 0.0283, -0.1865,  0.2529,  0.3732],
        [ 0.3199, -0.0360,  0.1801, -0.2992],
        [-0.1774, -0.0231, -0.4337,  0.2390]])
None
tensor([[-0.1566, -0.0125, -0.1530, -0.1648],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0268,  0.0018,  0.0030,  0.0055]])


### 4.2.2 初始化模型参数

PyTorch的 init 模块里提供了多种预设的初始化方法。在下面的例子中，将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零

In [12]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-7.1852e-03,  3.2517e-03, -4.1681e-05, -2.0226e-02],
        [ 1.8197e-02, -8.7433e-03,  4.5074e-03, -9.6044e-03],
        [-6.9399e-03,  9.5359e-03,  2.2113e-02, -1.4807e-03]])
2.weight tensor([[ 0.0151, -0.0081, -0.0224]])


下面使用常数来初始化权重参数

In [14]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 4.2.3 自定义初始化方法

实现一个自定义的初始化方法。在下面的例子里，我们令权重有一半概率初始化为0，有另一半概率初始化为[−10,−5][−10,−5]和[5,10][5,10]两个区间里均匀分布的随机数。

In [15]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-6.6450,  0.0000,  7.3079, -0.0000],
        [-5.6111,  0.0000, -0.0000,  0.0000],
        [-6.9541, -0.0000, -9.3860,  5.3725]])
2.weight tensor([[ 8.6604, -9.9607, -0.0000]])


还可以通过改变这些参数的data来改写模型参数值同时不会影响梯度

In [17]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


### 4.2.4 共享模型参数

多个层之间共享模型参数

In [18]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


在内存中，这两个线性层其实一个对象:

因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:

In [20]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


### 小结

- 有多种方法来访问、初始化和共享模型参数。
- 可以自定义初始化方法。

## 4.3 模型参数的延后初始化

## 4.4 自定义层

### 4.4.1 不含模型参数的自定义层

In [26]:
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()
    
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

构造更复杂的模型

In [31]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
y = net(torch.rand(4, 8))
y.mean().item()

Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): CenteredLayer()
)


### 4.4.2 含模型参数的自定义层

Parameter 类其实是 Tensor 的子类，如果一个 Tensor 是 Parameter，那么它会自动被添加到模型的参数列表里。所以在自定义含模型参数的层时，应该将参数定义成 Parameter，除了直接定义成 Parameter 类外，还可以使用 ParameterList 和 ParameterDict 分别定义参数的列表和字典。

In [27]:
class MyDense(nn.Module):
    def __init__(self):
        super(MyDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
net = MyDense()
print(net)

MyDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


## 4.5 读取和存储

### 4.5.1 读写Tensor

使用 save 函数和 load 函数分别存储和读取 Tensor

In [34]:
import torch
from torch import nn

x = torch.ones(3)
# 存储变量
torch.save(x, 'x.pt')
# 加载变量
x2 = torch.load('x.pt')
x2

tensor([1., 1., 1.])

### 4.5.2 读写模型

#### 4.5.2.1 state_dict

state_dict 是一个从参数名称隐射到参数 Tesnor 的字典对象。

In [35]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(3, 2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

net = MLP()
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.1691, -0.1295,  0.3571],
                      [ 0.0889,  0.0620, -0.2379]])),
             ('hidden.bias', tensor([-0.1728, -0.5719])),
             ('output.weight', tensor([[-0.5090,  0.1063]])),
             ('output.bias', tensor([-0.3109]))])

只有具有可学习参数的层(卷积层、线性层等)才有state_dict中的条目。优化器(optim)也有一个state_dict，其中包含关于优化器状态以及所使用的超参数的信息。

In [37]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [0, 1, 2, 3]}]}

#### 4.5.2.2 保存和加载模型

PyTorch 中保存和加载训练模型有两种常见的方法:

- 仅保存和加载模型参数(state_dict)；
- 保存和加载整个模型。

##### 4.5.2.2.1 保存和加载state_dict(推荐方式)

In [42]:
X = torch.randn(2, 3)
Y = net(X)

PATH = "./net.pt"
torch.save(net.state_dict(), PATH)

net2 = MLP()
net2.load_state_dict(torch.load(PATH))
Y2 = net2(X)
Y2 == Y

tensor([[True],
        [True]])

##### 4.5.2.2.2 保存和加载整个模型

保存
torch.save(model, PATH)

加载
model = torch.load(PATH)

### 小结

- 通过 save 函数和 load 函数可以很方便地读写 Tensor。
- 通过 save 函数和 load_state_dict 函数可以很方便地读写模型的参数。

## 4.6 GPU计算

### 4.6.1 计算设备

In [47]:
import torch
from torch import nn

# 查看 GPU 是否可用:
torch.cuda.is_available() # 输出 True
# 查看 GPU 数量
torch.cuda.device_count() # 输出 1
# 查看当前GPU索引号
torch.cuda.current_device() # 输出 0
# 根据索引号查看 GPU 名字
torch.cuda.get_device_name(0) # 输出 'GeForce GTX 1050'

'Quadro P600'

### 4.6.2 Tensor 的 GPU 计算

使用.cuda()可以将CPU上的Tensor转换（复制）到GPU上。如果有多块GPU，我们用.cuda(i)来表示第 ii 块GPU及相应的显存（ii从0开始）且cuda(0)和cuda()等价。

In [48]:
x = torch.tensor([1, 2, 3])
x = x.cuda(0)

可以通过Tensor的device属性来查看该Tensor所在的设备

In [49]:
x.device

device(type='cuda', index=0)

直接在创建的时候就指定设备。

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x = torch.tensor([1, 2, 3], device=device)
# or
x = torch.tensor([1, 2, 3]).to(device)
x

tensor([1, 2, 3], device='cuda:0')

需要注意的是，存储在不同位置中的数据是不可以直接进行计算的。即存放在CPU上的数据不可以直接与存放在GPU上的数据进行运算，位于不同GPU上的数据也是不能直接进行计算的。